In [2]:
import pandas as pd

In [3]:
messages = pd.read_csv('/Users/aravindmohan/Desktop/TFL/Datasets/spam_ham.csv')

In [4]:
messages


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will �_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [7]:
messages.drop(messages.columns[[2,3,4]],axis=1, inplace=True)

In [8]:
messages

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will �_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [9]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
ps=PorterStemmer()

In [13]:
corpus = []
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z0-9]', ' ', messages['v2'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)       

In [14]:
corpus

['go jurong point crazi avail bugi n great world la e buffet cine got amor wat',
 'ok lar joke wif u oni',
 'free entri 2 wkli comp win fa cup final tkt 21st may 2005 text fa 87121 receiv entri question std txt rate c appli 08452810075over18',
 'u dun say earli hor u c alreadi say',
 'nah think goe usf live around though',
 'freemsg hey darl 3 week word back like fun still tb ok xxx std chg send 1 50 rcv',
 'even brother like speak treat like aid patent',
 'per request mell mell oru minnaminungint nurungu vettam set callertun caller press 9 copi friend callertun',
 'winner valu network custom select receivea 900 prize reward claim call 09061701461 claim code kl341 valid 12 hour',
 'mobil 11 month u r entitl updat latest colour mobil camera free call mobil updat co free 08002986030',
 'gonna home soon want talk stuff anymor tonight k cri enough today',
 'six chanc win cash 100 20 000 pound txt csh11 send 87575 cost 150p day 6day 16 tsandc appli repli hl 4 info',
 'urgent 1 week free mem

## Bag of Words

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500, binary=True,ngram_range=(2,2))
X = cv.fit_transform(corpus).toarray()

In [19]:
X.shape

(5572, 2500)

NameError: name 'y' is not defined

In [23]:
y=pd.get_dummies(messages['v1'])
y=y.iloc[:,1].values

In [24]:
y

array([0, 0, 1, ..., 0, 0, 0], dtype=uint8)

In [30]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [27]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

#### Multinomial Naive Bayes

In [28]:
y_pred=spam_detect_model.predict(X_test)


In [31]:
score=accuracy_score(y_test,y_pred)
print(score)

0.9623318385650225


In [32]:
from sklearn.metrics import classification_report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98       989
           1       0.75      0.99      0.86       126

    accuracy                           0.96      1115
   macro avg       0.88      0.98      0.92      1115
weighted avg       0.97      0.96      0.96      1115



#### Random Forrest Classifier

In [35]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train,y_train)

RandomForestClassifier()

In [36]:
y_pred=classifier.predict(X_test)

In [37]:
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

0.9614349775784753
              precision    recall  f1-score   support

           0       1.00      0.96      0.98       984
           1       0.77      0.97      0.86       131

    accuracy                           0.96      1115
   macro avg       0.88      0.96      0.92      1115
weighted avg       0.97      0.96      0.96      1115



## Word2vec

In [38]:
import gensim

In [39]:
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [40]:
corpus = []
for i in range(0,len(messages)):
    review = re.sub('[^a-zA-Z0-9]', ' ', messages['v2'][i])
    review = review.lower()
    review = review.split()
    
    review = [lemmatizer.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)     

In [42]:
corpus[0]

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

In [43]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [44]:
words=[]
for sent in corpus:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [45]:
words

[['go',
  'jurong',
  'point',
  'crazy',
  'available',
  'bugis',
  'great',
  'world',
  'la',
  'buffet',
  'cine',
  'got',
  'amore',
  'wat'],
 ['ok', 'lar', 'joking', 'wif', 'oni'],
 ['free',
  'entry',
  'wkly',
  'comp',
  'win',
  'fa',
  'cup',
  'final',
  'tkts',
  'st',
  'may',
  'text',
  'fa',
  'receive',
  'entry',
  'question',
  'std',
  'txt',
  'rate',
  'apply',
  'over'],
 ['dun', 'say', 'early', 'hor', 'already', 'say'],
 ['nah', 'think', 'go', 'usf', 'life', 'around', 'though'],
 ['freemsg',
  'hey',
  'darling',
  'week',
  'word',
  'back',
  'like',
  'fun',
  'still',
  'tb',
  'ok',
  'xxx',
  'std',
  'chgs',
  'send',
  'rcv'],
 ['even', 'brother', 'like', 'speak', 'treat', 'like', 'aid', 'patent'],
 ['per',
  'request',
  'melle',
  'melle',
  'oru',
  'minnaminunginte',
  'nurungu',
  'vettam',
  'set',
  'callertune',
  'caller',
  'press',
  'copy',
  'friend',
  'callertune'],
 ['winner',
  'valued',
  'network',
  'customer',
  'selected',
  're

In [47]:
model=gensim.models.Word2Vec(words,window=5,min_count=2)

In [48]:
model

In [49]:
model.wv.index_to_key

['call',
 'get',
 'ur',
 'gt',
 'lt',
 'go',
 'ok',
 'free',
 'day',
 'know',
 'come',
 'like',
 'good',
 'time',
 'got',
 'text',
 'love',
 'want',
 'send',
 'need',
 'txt',
 'one',
 'today',
 'going',
 'home',
 'stop',
 'lor',
 'sorry',
 'see',
 'still',
 'mobile',
 'take',
 'back',
 'da',
 'reply',
 'think',
 'dont',
 'tell',
 'week',
 'phone',
 'hi',
 'new',
 'later',
 'pls',
 'please',
 'co',
 'msg',
 'make',
 'dear',
 'message',
 'night',
 'well',
 'say',
 'min',
 'thing',
 'much',
 'claim',
 'great',
 'oh',
 'hope',
 'hey',
 'number',
 'wat',
 'friend',
 'happy',
 'work',
 'give',
 'way',
 'yes',
 'www',
 'let',
 'prize',
 'right',
 'tomorrow',
 'already',
 'ask',
 'win',
 'cash',
 'said',
 'yeah',
 'really',
 'life',
 'amp',
 'tone',
 'babe',
 'im',
 'meet',
 'find',
 'morning',
 'service',
 'uk',
 'miss',
 'thanks',
 'last',
 'would',
 'com',
 'year',
 'anything',
 'lol',
 'nokia',
 'also',
 'care',
 'every',
 'feel',
 'pick',
 'keep',
 'sure',
 'urgent',
 'sent',
 'contact',


In [50]:
model.corpus_count

5565

In [51]:
model.epochs

5

In [73]:
model.wv.similar_by_word('hope')

[('love', 0.9996901750564575),
 ('day', 0.999686062335968),
 ('make', 0.9996287226676941),
 ('would', 0.9996178150177002),
 ('well', 0.9996119737625122),
 ('amp', 0.9996106624603271),
 ('need', 0.999607503414154),
 ('want', 0.9996060729026794),
 ('thing', 0.9996052384376526),
 ('great', 0.9995983839035034)]

In [74]:
import numpy as np

In [63]:
def avg_word2vec(doc):
    # remove out-of-vocabulary words
    #sent = [word for word in doc if word in model.wv.index_to_key]
    #print(sent)
    
    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)
                #or [np.zeros(len(model.wv.index_to_key))], axis=0)
        

In [64]:
from tqdm import tqdm


In [69]:
words[0]

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'got',
 'amore',
 'wat']

In [70]:
type(model.wv.index_to_key)


list

In [76]:
#apply for the entire sentences
X=[]
for i in tqdm(range(len(words))):
    print("Hello",i)
    X.append(avg_word2vec(words[i]))

  7%|▋         | 379/5565 [00:00<00:02, 1942.63it/s]

Hello 0
Hello 1
Hello 2
Hello 3
Hello 4
Hello 5
Hello 6
Hello 7
Hello 8
Hello 9
Hello 10
Hello 11
Hello 12
Hello 13
Hello 14
Hello 15
Hello 16
Hello 17
Hello 18
Hello 19
Hello 20
Hello 21
Hello 22
Hello 23
Hello 24
Hello 25
Hello 26
Hello 27
Hello 28
Hello 29
Hello 30
Hello 31
Hello 32
Hello 33
Hello 34
Hello 35
Hello 36
Hello 37
Hello 38
Hello 39
Hello 40
Hello 41
Hello 42
Hello 43
Hello 44
Hello 45
Hello 46
Hello 47
Hello 48
Hello 49
Hello 50
Hello 51
Hello 52
Hello 53
Hello 54
Hello 55
Hello 56
Hello 57
Hello 58
Hello 59
Hello 60
Hello 61
Hello 62
Hello 63
Hello 64
Hello 65
Hello 66
Hello 67
Hello 68
Hello 69
Hello 70
Hello 71
Hello 72
Hello 73
Hello 74
Hello 75
Hello 76
Hello 77
Hello 78
Hello 79
Hello 80
Hello 81
Hello 82
Hello 83
Hello 84
Hello 85
Hello 86
Hello 87
Hello 88
Hello 89
Hello 90
Hello 91
Hello 92
Hello 93
Hello 94
Hello 95
Hello 96
Hello 97
Hello 98
Hello 99
Hello 100
Hello 101
Hello 102
Hello 103
Hello 104
Hello 105
Hello 106
Hello 107
Hello 108
Hello 109
Hello 110


 15%|█▍        | 832/5565 [00:00<00:02, 2167.12it/s]

Hello 383
Hello 384
Hello 385
Hello 386
Hello 387
Hello 388
Hello 389
Hello 390
Hello 391
Hello 392
Hello 393
Hello 394
Hello 395
Hello 396
Hello 397
Hello 398
Hello 399
Hello 400
Hello 401
Hello 402
Hello 403
Hello 404
Hello 405
Hello 406
Hello 407
Hello 408
Hello 409
Hello 410
Hello 411
Hello 412
Hello 413
Hello 414
Hello 415
Hello 416
Hello 417
Hello 418
Hello 419
Hello 420
Hello 421
Hello 422
Hello 423
Hello 424
Hello 425
Hello 426
Hello 427
Hello 428
Hello 429
Hello 430
Hello 431
Hello 432
Hello 433
Hello 434
Hello 435
Hello 436
Hello 437
Hello 438
Hello 439
Hello 440
Hello 441
Hello 442
Hello 443
Hello 444
Hello 445
Hello 446
Hello 447
Hello 448
Hello 449
Hello 450
Hello 451
Hello 452
Hello 453
Hello 454
Hello 455
Hello 456
Hello 457
Hello 458
Hello 459
Hello 460
Hello 461
Hello 462
Hello 463
Hello 464
Hello 465
Hello 466
Hello 467
Hello 468
Hello 469
Hello 470
Hello 471
Hello 472
Hello 473
Hello 474
Hello 475
Hello 476
Hello 477
Hello 478
Hello 479
Hello 480
Hello 481
Hello 482


 23%|██▎       | 1274/5565 [00:00<00:01, 2184.07it/s]

...

Hello 4371
Hello 4372
Hello 4373
Hello 4374
Hello 4375
Hello 4376
Hello 4377
Hello 4378
Hello 4379
Hello 4380
Hello 4381
Hello 4382
Hello 4383
Hello 4384
Hello 4385
Hello 4386
Hello 4387
Hello 4388
Hello 4389
Hello 4390
Hello 4391
Hello 4392
Hello 4393
Hello 4394
Hello 4395
Hello 4396
Hello 4397
Hello 4398
Hello 4399
Hello 4400
Hello 4401
Hello 4402
Hello 4403
Hello 4404
Hello 4405
Hello 4406
Hello 4407
Hello 4408
Hello 4409
Hello 4410
Hello 4411
Hello 4412
Hello 4413
Hello 4414
Hello 4415
Hello 4416
Hello 4417
Hello 4418
Hello 4419
Hello 4420
Hello 4421
Hello 4422
Hello 4423
Hello 4424
Hello 4425
Hello 4426
Hello 4427
Hello 4428
Hello 4429
Hello 4430
Hello 4431
Hello 4432
Hello 4433
Hello 4434
Hello 4435
Hello 4436
Hello 4437
Hello 4438
Hello 4439
Hello 4440
Hello 4441
Hello 4442
Hello 4443
Hello 4444
Hello 4445
Hello 4446
Hello 4447
Hello 4448
Hello 4449
Hello 4450
Hello 4451
Hello 4452
Hello 4453
Hello 4454
Hello 4455
Hello 4456
Hello 4457
Hello 4458
Hello 4459
Hello 4460
Hello 4461

 91%|█████████ | 5071/5565 [00:02<00:00, 2173.30it/s]

Hello 4817
Hello 4818
Hello 4819
Hello 4820
Hello 4821
Hello 4822
Hello 4823
Hello 4824
Hello 4825
Hello 4826
Hello 4827
Hello 4828
Hello 4829
Hello 4830
Hello 4831
Hello 4832
Hello 4833
Hello 4834
Hello 4835
Hello 4836
Hello 4837
Hello 4838
Hello 4839
Hello 4840
Hello 4841
Hello 4842
Hello 4843
Hello 4844
Hello 4845
Hello 4846
Hello 4847
Hello 4848
Hello 4849
Hello 4850
Hello 4851
Hello 4852
Hello 4853
Hello 4854
Hello 4855
Hello 4856
Hello 4857
Hello 4858
Hello 4859
Hello 4860
Hello 4861
Hello 4862
Hello 4863
Hello 4864
Hello 4865
Hello 4866
Hello 4867
Hello 4868
Hello 4869
Hello 4870
Hello 4871
Hello 4872
Hello 4873
Hello 4874
Hello 4875
Hello 4876
Hello 4877
Hello 4878
Hello 4879
Hello 4880
Hello 4881
Hello 4882
Hello 4883
Hello 4884
Hello 4885
Hello 4886
Hello 4887
Hello 4888
Hello 4889
Hello 4890
Hello 4891
Hello 4892
Hello 4893
Hello 4894
Hello 4895
Hello 4896
Hello 4897
Hello 4898
Hello 4899
Hello 4900
Hello 4901
Hello 4902
Hello 4903
Hello 4904
Hello 4905
Hello 4906
Hello 4907

100%|██████████| 5565/5565 [00:02<00:00, 2139.68it/s]

Hello 5223
Hello 5224
Hello 5225
Hello 5226
Hello 5227
Hello 5228
Hello 5229
Hello 5230
Hello 5231
Hello 5232
Hello 5233
Hello 5234
Hello 5235
Hello 5236
Hello 5237
Hello 5238
Hello 5239
Hello 5240
Hello 5241
Hello 5242
Hello 5243
Hello 5244
Hello 5245
Hello 5246
Hello 5247
Hello 5248
Hello 5249
Hello 5250
Hello 5251
Hello 5252
Hello 5253
Hello 5254
Hello 5255
Hello 5256
Hello 5257
Hello 5258
Hello 5259
Hello 5260
Hello 5261
Hello 5262
Hello 5263
Hello 5264
Hello 5265
Hello 5266
Hello 5267
Hello 5268
Hello 5269
Hello 5270
Hello 5271
Hello 5272
Hello 5273
Hello 5274
Hello 5275
Hello 5276
Hello 5277
Hello 5278
Hello 5279
Hello 5280
Hello 5281
Hello 5282
Hello 5283
Hello 5284
Hello 5285
Hello 5286
Hello 5287
Hello 5288
Hello 5289
Hello 5290
Hello 5291
Hello 5292
Hello 5293
Hello 5294
Hello 5295
Hello 5296
Hello 5297
Hello 5298
Hello 5299
Hello 5300
Hello 5301
Hello 5302
Hello 5303
Hello 5304
Hello 5305
Hello 5306
Hello 5307
Hello 5308
Hello 5309
Hello 5310
Hello 5311
Hello 5312
Hello 5313

In [77]:
X


[array([-0.09123746,  0.24671991,  0.04697976,  0.00770951,  0.02126727,
        -0.41865385,  0.07743029,  0.52495694, -0.21224433, -0.16144815,
        -0.14626898, -0.35732746, -0.02890564,  0.09597632,  0.04006547,
        -0.18526323,  0.01888802, -0.25024635,  0.0010927 , -0.46579012,
         0.14873521,  0.08900166,  0.0070939 , -0.05052884, -0.16493954,
        -0.06119511, -0.17735624, -0.15358804, -0.23624991,  0.01881824,
         0.28370264,  0.03548132, -0.01763957, -0.1931717 , -0.11879093,
         0.2539837 ,  0.04059651, -0.21600455, -0.06827217, -0.47789013,
         0.03602482, -0.29084006, -0.0205885 ,  0.07130564,  0.2644157 ,
        -0.10352463, -0.18438816, -0.01218234,  0.25120282,  0.12424311,
         0.13520472, -0.20388071, -0.06399507, -0.09689995, -0.18421645,
         0.18000872,  0.1599846 , -0.06980742, -0.29979748,  0.01803588,
         0.07077574,  0.07597385, -0.04563154, -0.01554953, -0.29190174,
         0.21549366,  0.1037521 ,  0.13379714, -0.2

In [111]:
X_new=np.array(X)


In [112]:
words[0]

['go',
 'jurong',
 'point',
 'crazy',
 'available',
 'bugis',
 'great',
 'world',
 'la',
 'buffet',
 'cine',
 'got',
 'amore',
 'wat']

In [113]:
#Every word is converted
X_new[0]


array([-0.09123746,  0.24671991,  0.04697976,  0.00770951,  0.02126727,
       -0.41865385,  0.07743029,  0.52495694, -0.21224433, -0.16144815,
       -0.14626898, -0.35732746, -0.02890564,  0.09597632,  0.04006547,
       -0.18526323,  0.01888802, -0.25024635,  0.0010927 , -0.46579012,
        0.14873521,  0.08900166,  0.0070939 , -0.05052884, -0.16493954,
       -0.06119511, -0.17735624, -0.15358804, -0.23624991,  0.01881824,
        0.28370264,  0.03548132, -0.01763957, -0.1931717 , -0.11879093,
        0.2539837 ,  0.04059651, -0.21600455, -0.06827217, -0.47789013,
        0.03602482, -0.29084006, -0.0205885 ,  0.07130564,  0.2644157 ,
       -0.10352463, -0.18438816, -0.01218234,  0.25120282,  0.12424311,
        0.13520472, -0.20388071, -0.06399507, -0.09689995, -0.18421645,
        0.18000872,  0.1599846 , -0.06980742, -0.29979748,  0.01803588,
        0.07077574,  0.07597385, -0.04563154, -0.01554953, -0.29190174,
        0.21549366,  0.1037521 ,  0.13379714, -0.29916024,  0.31

In [114]:
y = messages[list(map(lambda x: len(x)>0 ,corpus))]


In [115]:
y

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will �_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [116]:
y=pd.get_dummies(y['v1'])
y=y.iloc[:,1].values

In [117]:
y

array([0, 0, 1, ..., 0, 0, 0], dtype=uint8)

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size = 0.20, random_state = 0)

In [119]:
y.shape

(5565,)

In [120]:
from sklearn.ensemble import RandomForestClassifier
classifier=RandomForestClassifier()
classifier.fit(X_train,y_train)

ValueError: setting an array element with a sequence.

In [121]:
y_pred=classifier.predict(X_test)

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [99]:
print(accuracy_score(y_pred,y_test))
print(classification_report(y_pred,y_test))

0.9614349775784753
              precision    recall  f1-score   support

           0       1.00      0.96      0.98       984
           1       0.77      0.97      0.86       131

    accuracy                           0.96      1115
   macro avg       0.88      0.96      0.92      1115
weighted avg       0.97      0.96      0.96      1115

